### Methodology

* Instance type ⇒ c3.8xlarge
* OS ⇒ Centos 7

### Setup

#### Kernel Modules

Make sure the relevant crypto modules are loaded into the kernel (if available for your platform).

```bash
$ for m in aesni_intel sha512-ssse3 sha1-ssse3 sha256 sha256-sse; do sudo modprobe $m; done
```

#### Software Deps

```bash
$ sudo yum install -y python-devel python-pip libffi-devel nasm haveged sysstat zlib-devel
$ sudo yum groupinstall -y "Development tools"
$ curl -O http://download.gna.org/cryptodev-linux/cryptodev-linux-1.8.tar.gz
$ tar xf cryptodev-linux-1.8.tar.gz
$ cd cryptodev-linux-1.8/
$ make && sudo make install
$ sudo sh -c 'echo "cryptodev" > /etc/modules-load.d/cryptodev.conf'
$ sudo modprobe cryptodev
$ curl -O ftp://ftp.openssl.org/source/openssl-fips-2.0.12.tar.gz
$ tar xf openssl-fips-2.0.12.tar.gz
$ cd openssl-fips-2.0.12
$ ./config --prefix=/usr --openssldir=/etc/pki/tls zlib threads -DHAVE_CRYPTODEV -DGHASH_ASM -DWHIRLPOOL_ASM -DPURIFY -DOPENSSL_IA32_SSE2 -DOPENSSL_BN_ASM_MONT -DOPENSSL_BN_ASM_MONT5 -DOPENSSL_BN_ASM_GF2m -DSHA1_ASM -DSHA256_ASM -DSHA512_ASM -DMD5_ASM -DAES_ASM -DVPAES_ASM -DBSAES_ASM -DOPENSSL_PIC -DZLIB -DOPENSSL_THREADS -D_REENTRANT -DDSO_DLFCN -DHAVE_DLFCN_H -DKRB5_MIT  -DL_ENDIAN -DTERMIO -fPIC  -m64 -Wall -O2 -g -pipe -Wall -Wp,-D_FORTIFY_SOURCE=2 -fexceptions -fstack-protector-strong    -m64 -mtune=native -Wa,--noexecstack  -grecord-gcc-switches && make && sudo make install
$ curl -O https://www.openssl.org/source/openssl-1.0.2g.tar.gz
$ tar xf openssl-1.0.2g.tar.gz
$ cd openssl-1.0.2g
$ ./config --prefix=/usr --with-fipsdir=/usr --with-fipslibdir=/usr/lib64/ --openssldir=/etc/pki/tls fips zlib threads -DHAVE_CRYPTODEV -DGHASH_ASM -DWHIRLPOOL_ASM -DPURIFY -DOPENSSL_IA32_SSE2 -DOPENSSL_BN_ASM_MONT -DOPENSSL_BN_ASM_MONT5 -DOPENSSL_BN_ASM_GF2m -DSHA1_ASM -DSHA256_ASM -DSHA512_ASM -DMD5_ASM -DAES_ASM -DVPAES_ASM -DBSAES_ASM -fPIC -DOPENSSL_PIC -DZLIB -DOPENSSL_THREADS -D_REENTRANT -DDSO_DLFCN -DHAVE_DLFCN_H -DKRB5_MIT -m64 -DL_ENDIAN -DTERMIO -Wall -O2 -g -pipe -Wall -Wp,-D_FORTIFY_SOURCE=2 -fexceptions -fstack-protector-strong --param=ssp-buffer-size=4 -grecord-gcc-switches   -m64 -mtune=native -Wa,--noexecstack && make depend && make && sudo make install
$ sudo pip install -U ndg-httpsclient pyasn1 pyOpenSSL awscli
$ cp $(which aws) ~/aws
$ sudo mkdir /media/ramfs
$ sudo mount -t ramfs -o size=40g ramfs /media/ramfs
$ haveged -n 30g -f /media/ramfs/data/data.30g
$ cat << EOF >> /etc/fstab
tmpfs   /media/ramfs    tmpfs   nodev,nosuid,noexec,nodiratime,size=40g 0       0
EOF
```

#### AWS CLI

Add the following to `~/aws` before the `import awscli.clidriver` line.

```python
import botocore.vendored.requests.packages.urllib3.util.ssl_
botocore.vendored.requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = "AES128-SHA" 
#botocore.vendored.requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = "aNULL" # If this is working, https should not work if this is enabled
```

---

### Test Script(s)


```bash
#!/bin/env bash
# This will run 10 concurrent uploads for each of the different scenarios, SSE, SSL, and plaintext.
# It will run each test 5 times.

for i in 1 2 3 4 5; do
    ####################### SSE ###############################
    echo "doing sse $i"
    sar -p 1 > "async_yes_ssl_yes_sse-cpu.${i}.sar.txt" &
    sar_cpu=$!

    sar -p -n DEV 1 > "async_yes_ssl_yes_sse-network.${i}.sar.txt" &
    sar_network=$!

    sleep 1
    
    jobs=""
    for j in 0 1 2 3 4 5 6 7 8 9; do
	~/aws s3 cp /media/ramfs/data/data.30g "s3://psoutham-encryption-test/output/${j}_yes_sse_async_${i}.30g"  --storage-class REDUCED_REDUNDANCY --endpoint-url https://s3-us-west-2.amazonaws.com --sse "aws:kms" --sse-kms-key-id a9de2efa-8278-420c-86ea-272881fcde8d &
	jobs="${jobs} $!"
	usleep 300000 
    done
    echo "waiting for ${jobs}"
    wait ${jobs}
    sleep 1
    kill "${sar_cpu}" "${sar_network}"

    ####################### SSL ###############################

    echo "doing ssl $i"
    sar -p 1 > "async_yes_ssl_no_sse-cpu.${i}.sar.txt" &
    sar_cpu=$!

    sar -p -n DEV 1 > "async_yes_ssl_no_sse-network.${i}.sar.txt" &
    sar_network=$!

    sleep 1
    
    jobs=""
    for j in 0 1 2 3 4 5 6 7 8 9; do
	~/aws s3 cp /media/ramfs/data/data.30g "s3://psoutham-encryption-test/output/${j}_yes_ssl_async_${i}.30g"  --storage-class REDUCED_REDUNDANCY --endpoint-url https://s3-us-west-2.amazonaws.com  &
	jobs="${jobs} $!"
	usleep 300000 
    done
    echo "waiting for ${jobs}"
    wait ${jobs}
    sleep 1
    kill "${sar_cpu}" "${sar_network}"

    ####################### PLAINTEXT ###############################

    echo "doing plaintext $i"
    sar -p 1 > "async_no_ssl_no_sse-cpu.${i}.sar.txt" &
    sar_cpu=$!

    sar -p -n DEV 1 > "async_no_ssl_no_sse-network.${i}.sar.txt" &
    sar_network=$!

    sleep 1
    
    jobs=""
    for j in 0 1 2 3 4 5 6 7 8 9; do
	~/aws s3 cp /media/ramfs/data/data.30g "s3://psoutham-encryption-test/output/${j}_no_ssl_async_${i}.30g"  --storage-class REDUCED_REDUNDANCY --endpoint-url http://s3-us-west-2.amazonaws.com &
	jobs="${jobs} $!"
	usleep 300000 
    done
    echo "waiting for ${jobs}"
    wait ${jobs}
    sleep 1
    kill "${sar_cpu}" "${sar_network}"
done
```